In [1]:
import numpy
import scipy.stats

In [2]:
N = 2

# Gaussian expectation vector
Mu = numpy.zeros(N)

# Gaussian variance-covariance matrix
rho = 0.
Sigma = numpy.array([[1., rho], [rho, 1.]])

print "Mu = %s" % Mu
print "Sigma = \n %s" % Sigma

ctp = 0.

Mu = [ 0.  0.]
Sigma = 
 [[ 1.  0.]
 [ 0.  1.]]


# Link between paper and code

\begin{equation*}
\ell(x) = \sum_k x_k + \frac{1}{2} \sum_k (x_k^+)^2
\end{equation*}

We need to solve the following system:
    
\begin{equation*}
\begin{cases}
\lambda \left[ 1
+ \mathbb{E} \left( \left(X_k - m_k^* \right)^+ \right) 
\right] = 1 \text{ for all $k$} \\
\sum_k \mathbb{E} \left( X_k - m_k^* \right) 
+ \sum_k \frac{1}{2} \mathbb{E} \left( \left[ \left( X_k - m_k^* \right)^+ \right]^2 \right) 
= c
\end{cases}
\end{equation*}

Definitions from the paper:

\begin{equation*}
\begin{cases}
\lambda \left[ 1
+ f_k \left( m_k^* \right) 
\right] = 1 \text{ for all $k$} \\
\sum_k \left( e_k \left( m_k^* \right) 
+ \frac{1}{2} g_k \left( m_k^* \right)  \right) 
= c
\end{cases}
\end{equation*}

In [3]:
def e(index, x):
    mu = Mu[index]
    
    return mu - x

In [4]:
def f(index, x):
    mu = Mu[index]
    sigma = numpy.sqrt(Sigma[index, index])
    
    first_term = (mu - x) * scipy.stats.norm.cdf((mu - x)/sigma)
    sec_term = sigma / numpy.sqrt(2 * numpy.pi) * numpy.exp(-.5 * ((mu - x)/sigma)**2)
    
    return first_term + sec_term

In [5]:
def g(index, x):
    mu = Mu[index]
    sigma2 = Sigma[index, index]
    sigma = numpy.sqrt(sigma2)
    
    first_term = ((mu - x)**2 + sigma2) * scipy.stats.norm.cdf((mu - x)/sigma)
    sec_term = (mu - x)*sigma * numpy.exp(-.5 * ((mu - x)/sigma)**2) / numpy.sqrt(2 * numpy.pi)
    
    return first_term + sec_term

In [6]:
class WrapperFunc(object):
    def __init__(self, func, indexes):
        self._i = indexes
        self._f = func
        
    def __call__(self, *x):
        tmp_args = self._i + list(x)
        return self._f(*tmp_args)

## Put the variables and functions to the remote engines

In [7]:
def init_funcs():
    funcs = dict()
    
    for k in range(N):
        ek = WrapperFunc(e, [k])
        fk = WrapperFunc(f, [k])
        gk = WrapperFunc(g, [k])
    
        funcs[k] = {"e": ek, "f": fk, "g": gk}
        
    return funcs

In [8]:
def prepare_parameterized_funcs(dict_funcs):
    res = dict()
        
    for k, v in dict_funcs.iteritems():
        for kk, vv in v.iteritems():
            if isinstance(vv, WrapperFunc):
                key = "%s%s" % (kk, k)
                res[key] = (vv, k)
        
    return res

In [9]:
def run_par_func(param):
    f = param[0]
    vm = []
    for i_m in param[1:]:
        vm.append(M[i_m])
    
    return f(*vm)

In [10]:
funcs = init_funcs()
flatten_funcs = prepare_parameterized_funcs(funcs)

In [11]:
from timeit import default_timer as timer

In [12]:
from scipy.optimize import fsolve

def optimize(c, **kwargs):
    global N
    
    __results = {key: dict() for key in "efg"}
        
    def fill_results(lbl, res):
        fname = lbl[0]
        k = int(lbl[-1])

        if len(lbl) == 3:
            j = int(lbl[1])
            if k not in __results[fname]:
                __results[fname][k] = dict()
                
            __results[fname][k][j] = res
        else:
            __results[fname][k] = res
    
    def objective(v_x):
        _lambda = v_x[-1]
        
        # Local version
        global M
        M = v_x
        f_res = map(run_par_func, flatten_funcs.values())
        
        for lbl, fx in zip(flatten_funcs.keys(), f_res):
            fill_results(lbl, fx)
            
        last = 0.
        res = numpy.zeros(N + 1)
        
        for k in range(N):
            res_k = 1 + __results["f"][k]
            res[k] = (_lambda * res_k - 1.)**2
            
            last += __results["e"][k] + .5 * __results["g"][k]
                    
        res[-1] = (last - c)**2
        return res
    
    guess = numpy.zeros(N + 1)
    if 'guess' in kwargs and kwargs['guess'] is not None:
        guess = kwargs.pop('guess')
        
    return fsolve(objective, guess, **kwargs) 

In [13]:
c = 1.

guess = numpy.ones(N+1)

In [14]:
tic = timer()
optim_result = optimize(c, guess=guess, maxfev=1000, full_output=True)
toc_m_tic = timer() - tic

print optim_result[-1]

The solution converged.


In [15]:
m_star = optim_result[0][:-1]
lb_star = optim_result[0][-1]
cto = toc_m_tic

print "Time of optimization: %.2f sec" % (ctp)
print "m star = %s" % m_star
print "lambda star = %s" % lb_star

Time of optimization: 0.00 sec
m star = [-0.17310526 -0.17310526]
lambda star = 0.670485184632


In [16]:
l_N = optim_result[1]["nfev"]

print "l(2) = %i" % l_N
print "function evaluated at the output = %s" % optim_result[1]["fvec"]

l(2) = 59
function evaluated at the output = [  3.96564965e-21   3.96566363e-21   1.51409072e-21]


### Just need to copy the results in Excel:

In [17]:
print "%f \n %f \n %i \n %.2f \n %.2f" % (m_star[0], lb_star, l_N, ctp, cto)

-0.173105 
 0.670485 
 59 
 0.00 
 0.06
